# Data Processing

First we import all the packages we will need

In [8]:
##import packages here
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
import re
from dateutil.relativedelta import relativedelta

# We'll use this later
from numpy import meshgrid

## This sets the plot style
## to have a grid on a white background
import seaborn as sns
sns.set_style("whitegrid")

### Now we put various datasets we obtained into dataframes

Above each code snippet is a brief description of the dataset we are processing.

##### NASDAQ_df

This dataframe contains historical data for the NASDAQ.

The columns are: Date, Open, High, Low, Close, Adj Close, Volume.

The dates range from Feb 5, 1971 to May 7 2021. The data is recorded daily.

I do not know where we got this dataset.

In [9]:
##import prices from ^IXIC.csv
##has data since 2/5/71
NASDAQ_df = pd.read_csv('../data/NASDAQ-all-time-daily.csv')

##drop NaN rows
NASDAQ_df.dropna()
##no need to drop more rows

##convert the dates to datetime
NASDAQ_df['Date'] = pd.to_datetime(NASDAQ_df['Date'])

NASDAQ_df = NASDAQ_df.set_index('Date')
NASDAQ_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1971-02-05,100.000000,100.000000,100.000000,100.000000,100.000000,0
1971-02-08,100.839996,100.839996,100.839996,100.839996,100.839996,0
1971-02-09,100.760002,100.760002,100.760002,100.760002,100.760002,0
1971-02-10,100.690002,100.690002,100.690002,100.690002,100.690002,0
1971-02-11,101.449997,101.449997,101.449997,101.449997,101.449997,0
...,...,...,...,...,...,...
2021-05-03,14031.769531,14042.120117,13881.509766,13895.120117,13895.120117,4718850000
2021-05-04,13774.509766,13795.570313,13485.589844,13633.500000,13633.500000,5852850000
2021-05-05,13731.129883,13753.049805,13553.929688,13582.419922,13582.419922,4517100000


#### SP500_df

This dataframe contains historical data for the S&P 500 index.

The columns are: Date, Open, High, Low, Close.

The dates range from Jan 3, 1978 to May 14, 2021. The data is recorded daily.

I downloaded this dataset from the Wall Street Journal.

In [10]:
##import SP500 open/high/low/close
##has data from 1950 to 2015, add data from 2015-2021?
SP500_df = pd.read_csv('../data/sp-HistoricalPrices-wsj.csv')
#, names=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
SP500_df

##didn't need to drop any rows

##change the date column to be in datetime
SP500_df['Date'] = pd.to_datetime(SP500_df['Date'])

##set the index to be the date column
SP500_df = SP500_df.set_index('Date')

SP500_df

,Open,High,Low,Close
Date,,,,
2021-05-14,4129.58,4183.13,4129.58,4173.85
2021-05-13,4074.99,4131.58,4074.99,4112.50
2021-05-12,4130.55,4134.73,4056.88,4063.04
2021-05-11,4150.34,4162.04,4111.53,4152.10
2021-05-10,4228.29,4236.39,4188.13,4188.43
...,...,...,...,...
1978-01-09,90.64,91.52,90.01,90.64
1978-01-06,91.62,92.66,91.05,91.62
1978-01-05,92.74,94.53,92.51,92.74


#### pe_df

This dataframe contains historical price-to-earnings ratio for the S&P 500.

The columns are: date, value.

The dates range from Dec 1927 to Dec 2020. The data is recorded monthly.

I do not know where we got this dataset.

##### daily data

We added a column called daily_value, and rows for each day. The daily_value is equal to the value in the month the date occurs in.

In [11]:
##import pe ratio from sp-500-pe-ratio-price-to-earnings-chart.csv
##has data since 1928 for each MONTH
file_pe = open('../data/sp-500-pe-ratio-price-to-earnings-chart.csv')
pe_df = pd.read_csv(file_pe, names=['date', 'value'], skiprows=[0,1,2,3,4,5,6,7])
file_pe.close()

##drop NaN rows
pe_df.dropna()
##drop header rows
pe_df = pe_df[3:]

##change date column to be in datetime format
pe_df['date'] = pd.to_datetime(pe_df['date'])

##set the index column to be the date column
pe_df = pe_df.set_index('date')

pe_df

,value
date,
1927-12-01,15.9099
1928-01-01,14.8898
1928-02-01,14.6271
1928-03-01,16.3390
1928-04-01,15.8000
...,...
2020-08-01,35.6266
2020-09-01,34.2290
2020-10-01,33.2820


#### bond_df

This dataset contains the historical bond yields for US treasury bonds.

The columns are: Date, Value.

The dates range from Jan 1, 1962 to May 6, 2021. The data is recorded daily.

I do not know where we got this dataset.

In [12]:
##import bond yield from 10-year-treasury-bond-rate-yield-chart.csv
##has data since 1962
file_bond = open('../data/10-year-treasury-bond-rate-yield-chart.csv')
bond_df = pd.read_csv(file_bond, names=['date', 'value'])
file_bond.close()

##drop NaN rows
bond_df = bond_df.dropna()
##drop header rows
bond_df = bond_df[1:]

##change the date column to be in datetime format
bond_df['date'] = pd.to_datetime(bond_df['date'])

##make the date column the index
bond_df = bond_df.set_index('date')

bond_df

,value
date,
1962-01-02,4.0600
1962-01-03,4.0300
1962-01-04,3.9900
1962-01-05,4.0200
1962-01-08,4.0300
...,...
2021-04-30,1.6500
2021-05-03,1.6300
2021-05-04,1.6100


#### ffund_df

federal funding rate dataset

In [13]:
##import the dataset to a csv
ffund_df = pd.read_csv('../data/fedfundrate.csv')

##convert the dates to datetime
ffund_df['DATE'] = pd.to_datetime(ffund_df['DATE'])

##make the dates the index column
ffund_df = ffund_df.set_index('DATE')

ffund_df

,FEDFUNDS
DATE,
1954-07-01,0.80
1954-08-01,1.22
1954-09-01,1.07
1954-10-01,0.85
1954-11-01,0.83
...,...
2020-12-01,0.09
2021-01-01,0.09
2021-02-01,0.08


#### ftass_df

the dataset with fed total assets

In [14]:
##import the dataset to a dataframe
ftass_df = pd.read_csv('../data/fedtotalassets.csv')

##convert the dates to datetime
ftass_df['DATE'] = pd.to_datetime(ftass_df['DATE'])

##make the date the index column
ftass_df = ftass_df.set_index('DATE')

ftass_df

,RESPPANWW
DATE,
2002-12-18,720761.0
2002-12-25,733136.0
2003-01-01,732202.0
2003-01-08,724902.0
2003-01-15,721325.0
...,...
2021-04-14,7793104.0
2021-04-21,7820948.0
2021-04-28,7780962.0


#### div_df

This dataset records the dividend rate per share average for the S&P 500

I got this data from quandl.

In [15]:
file_div = open('../data/SP500_DIV_YIELD_MONTH.csv')
div_df = pd.read_csv(file_div)
file_div.close()

##convert the dates to datetime
div_df['Date'] = pd.to_datetime(div_df['Date'])

##make the date column the index
div_df = div_df.set_index('Date')

div_df

,Value
Date,
2021-05-31,1.37
2021-04-30,1.41
2021-04-01,1.45
2021-03-31,1.47
2021-03-01,1.49
...,...
1871-05-31,5.35
1871-04-30,5.49
1871-03-31,5.64


#### eps_df

In [16]:
file_eps = open('../data/sp500eps.csv')
eps_df = pd.read_csv(file_eps, names=['Date', 'Value'])
file_eps.close()

##drop NaN rows
eps_df.dropna()

##drop header rows
eps_df = eps_df[1:]

##change date column to be in datetime format
eps_df['Date'] = pd.to_datetime(eps_df['Date'])

##set the index column to be the date column
eps_df = eps_df.set_index('Date')

#eps_df = eps_df.sort_values(['Date'])

eps_df

,Value
Date,
2020-12-31,95.72
2020-11-30,97.2
2020-10-31,98.53
2020-09-30,99.95
2020-08-31,100.44
...,...
1871-05-31,8.63
1871-04-30,8.44
1871-03-31,8.13


In [17]:
def nearest_week(d,items):
    distances = pd.DataFrame({'date' : items, 'distance' :[(d - w).total_seconds() for w in items]})
    distances = distances.set_index('distance')
    posdist = [dist for dist in distances.index if dist > 0]
    ##if there are no positive distances
    if len(posdist) == 0:
        abs_dist = abs(distances.index)
        ##then the smallest distance must be negative
        return distances.loc[-min(abs_dist)]['date']
    ##if there is a positive distance
    else:
        return distances.loc[min(posdist)]['date']

In [18]:
##get the dates that occur in every dataset
sp500_dates = SP500_df.index
pe_dates = pd.date_range(pe_df.index[0], pe_df.index[-1] + pd.Timedelta(weeks=4,days=2))
bond_dates = bond_df.index
ffund_dates = pd.date_range(ffund_df.index[0], ffund_df.index[-1] + pd.Timedelta(weeks=4,days=1))
eps_dates = pd.date_range(eps_df.index[-1], eps_df.index[3])

##get the dates that occur in every dataset
total_dates = pe_dates & sp500_dates & bond_dates & ffund_dates & eps_dates
total_dates

<ipython-input-18-7283afb28aab>:9: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  total_dates = pe_dates & sp500_dates & bond_dates & ffund_dates & eps_dates


DatetimeIndex(['1978-01-03', '1978-01-04', '1978-01-05', '1978-01-06',
               '1978-01-09', '1978-01-10', '1978-01-11', '1978-01-12',
               '1978-01-13', '1978-01-16',
               ...
               '2020-09-17', '2020-09-18', '2020-09-21', '2020-09-22',
               '2020-09-23', '2020-09-24', '2020-09-25', '2020-09-28',
               '2020-09-29', '2020-09-30'],
              dtype='datetime64[ns]', length=10677, freq=None)

In [19]:
##make a dataframe with all the information
total_df = SP500_df.copy()

##drop dates that don't occur in all datasets
drop_dates = [d for d in total_df.index if d not in total_dates]
total_df = total_df.drop(drop_dates)

##add the pe column
#total_df['pe'] = [pe_df.loc[d.replace(day=1), 'value'] for d in total_df.index]

##add the eps column
total_df['eps'] = [eps_df.loc[d + relativedelta(day=31, months=-1)]['Value'] for d in total_df.index]

##add the bond column
total_df['bond'] = bond_df['value']

##add the 'fed funds' column
total_df['fed funds'] = [ffund_df.loc[d.replace(day=1), 'FEDFUNDS'] for d in total_df.index]

##add total fed assets
#total_df['fed total assets'] = [ftass_df.loc[nearest_week(d, ftass_df.index)]['RESPPANWW'] for d in total_df.index]

total_df

,Open,High,Low,Close,eps,bond,fed funds
Date,,,,,,,
2020-09-30,3341.21,3393.56,3340.47,3363.00,100.44,0.6900,0.09
2020-09-29,3350.92,3357.92,3327.54,3335.47,100.44,0.6600,0.09
2020-09-28,3333.90,3360.74,3332.91,3351.60,100.44,0.6700,0.09
2020-09-25,3236.66,3306.88,3228.44,3298.46,100.44,0.6600,0.09
2020-09-24,3226.14,3278.70,3209.45,3246.59,100.44,0.6700,0.09
...,...,...,...,...,...,...,...
1978-01-09,90.64,91.52,90.01,90.64,46.45,8.0100,6.70
1978-01-06,91.62,92.66,91.05,91.62,46.45,7.8500,6.70
1978-01-05,92.74,94.53,92.51,92.74,46.45,7.8300,6.70


In [20]:
##add total fed assets
total_df['fed total assets'] = [ftass_df.loc[nearest_week(d, ftass_df.index)]['RESPPANWW'] for d in total_df.index]

In [23]:
##add dividend yield
total_df['div'] = [div_df.loc[nearest_week(d, div_df.index)]['Value'] for d in total_df.index]

In [24]:
total_df

,Open,High,Low,Close,eps,bond,fed funds,fed total assets,div
Date,,,,,,,,,
2020-09-30,3341.21,3393.56,3340.47,3363.00,100.44,0.6900,0.09,7093161.0,1.69
2020-09-29,3350.92,3357.92,3327.54,3335.47,100.44,0.6600,0.09,7093161.0,1.69
2020-09-28,3333.90,3360.74,3332.91,3351.60,100.44,0.6700,0.09,7093161.0,1.69
2020-09-25,3236.66,3306.88,3228.44,3298.46,100.44,0.6600,0.09,7093161.0,1.69
2020-09-24,3226.14,3278.70,3209.45,3246.59,100.44,0.6700,0.09,7093161.0,1.69
...,...,...,...,...,...,...,...,...,...
1978-01-09,90.64,91.52,90.01,90.64,46.45,8.0100,6.70,720761.0,4.98
1978-01-06,91.62,92.66,91.05,91.62,46.45,7.8500,6.70,720761.0,4.98
1978-01-05,92.74,94.53,92.51,92.74,46.45,7.8300,6.70,720761.0,4.98


In [25]:
#export to csv
total_df.to_csv("../data/total_df.csv")